In [25]:
import cv2
import math
from cvzone.HandTrackingModule import HandDetector
import numpy as np
import os
import time
import shutil  
import sys  

# Constants
OFFSET = 25
IMG_SIZE = 128
DATA_FOLDER = "Data"
MAX_IMAGES = 200
DELAY_BETWEEN_NUMBERS = 2  # Seconds to wait between number collections

def reset_data():
    if os.path.exists(DATA_FOLDER):
        shutil.rmtree(DATA_FOLDER)  
    os.makedirs(DATA_FOLDER)  
    for i in range(1, 10):
        os.makedirs(os.path.join(DATA_FOLDER, str(i)))  

def display_keybinds(frame):
    x, y, w, h = frame.shape[1] - 200, 10, 190, 100  
    cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 255, 255), -1)  
    
    cv2.putText(frame, "Keybinds:", (x + 10, y + 20), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
    cv2.putText(frame, "S: Save Image", (x + 10, y + 40), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
    cv2.putText(frame, "Q: Quit Program", (x + 10, y + 60), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
    cv2.putText(frame, "R: Restart Collection", (x + 10, y + 80), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

def main():
    reset_data()  

    capture = cv2.VideoCapture(0)
    detector = HandDetector(maxHands=1)

    current_number = 1
    image_count = 0
    waiting_for_next = False
    wait_start_time = None  

    while True:
        success, frame = capture.read()
        if not success:
            continue

        # Detect hands
        hands, frame_with_markings = detector.findHands(frame, draw=True)  

        img_black = None
        if hands:
            hand = hands[0]
            x, y, w, h = hand["bbox"]

            img_height, img_width = frame.shape[:2]
            x1, y1 = max(0, x - OFFSET), max(0, y - OFFSET)
            x2, y2 = min(img_width, x + w + OFFSET), min(img_height, y + h + OFFSET)

            if x2 > x1 and y2 > y1:
                hand_crop = frame[y1:y2, x1:x2]
                img_black = np.zeros((IMG_SIZE, IMG_SIZE, 3), np.uint8)

                crop_h, crop_w = hand_crop.shape[:2]
                aspect_ratio = crop_h / crop_w

                if aspect_ratio > 1:
                    resized_w = math.ceil((IMG_SIZE / crop_h) * crop_w)
                    resized_img = cv2.resize(hand_crop, (resized_w, IMG_SIZE))
                    gap = (IMG_SIZE - resized_w) // 2
                    img_black[:, gap:gap + resized_w] = resized_img
                else:
                    resized_h = math.ceil((IMG_SIZE / crop_w) * crop_h)
                    resized_img = cv2.resize(hand_crop, (IMG_SIZE, resized_h))
                    gap = (IMG_SIZE - resized_h) // 2
                    img_black[gap:gap + resized_h, :] = resized_img

                if img_black is not None:
                    cv2.imshow("HandSign", img_black)

        # Handle waiting between number collections
        if waiting_for_next:
            if wait_start_time is None:
                wait_start_time = time.time()
            
            elapsed_time = time.time() - wait_start_time
            remaining_time = max(0, DELAY_BETWEEN_NUMBERS - elapsed_time)
            
            # Status text with countdown
            cv2.rectangle(frame_with_markings, (5, 5), (300, 50), (255, 255, 255), -1)
            countdown_text = f"Next {current_number + 1} in {remaining_time:.1f}s"
            cv2.putText(frame_with_markings, countdown_text, (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)
            
            # Move to next number after delay
            if elapsed_time >= DELAY_BETWEEN_NUMBERS:
                current_number += 1
                image_count = 0
                waiting_for_next = False
                wait_start_time = None
                print(f"Now collecting for {current_number}")
                sys.stdout.flush()
        else:
            cv2.rectangle(frame_with_markings, (5, 5), (300, 50), (255, 255, 255), -1)
            cv2.putText(frame_with_markings, f"Collecting {current_number}", (10, 30), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 0), 2)

        # Display keybinds
        display_keybinds(frame_with_markings)  
        cv2.imshow("AirTacToe", frame_with_markings)

        key = cv2.waitKey(1)
        if key == ord('q'):
            break

        if key == ord('r'):
            print("Restarting...")
            sys.stdout.flush()
            reset_data()
            current_number = 1
            image_count = 0
            waiting_for_next = False
            wait_start_time = None

        if key == ord('s') and img_black is not None and not waiting_for_next:
            save_path = os.path.join(DATA_FOLDER, str(current_number), f"{image_count + 1}.jpg")
            cv2.imwrite(save_path, img_black)
            print(f"Saved: {save_path}")
            sys.stdout.flush()

            image_count += 1

            if image_count >= MAX_IMAGES:
                if current_number < 9:
                    waiting_for_next = True
                else:
                    print("Data collection complete!")
                    sys.stdout.flush()
                    break

    capture.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Saved: Data\1\1.jpg
Saved: Data\1\2.jpg
Saved: Data\1\3.jpg
Saved: Data\1\4.jpg
Saved: Data\1\5.jpg
Saved: Data\1\6.jpg
Saved: Data\1\7.jpg
Saved: Data\1\8.jpg
Saved: Data\1\9.jpg
Saved: Data\1\10.jpg
Saved: Data\1\11.jpg
Saved: Data\1\12.jpg
Saved: Data\1\13.jpg
Saved: Data\1\14.jpg
Saved: Data\1\15.jpg
Saved: Data\1\16.jpg
Saved: Data\1\17.jpg
Saved: Data\1\18.jpg
Saved: Data\1\19.jpg
Saved: Data\1\20.jpg
Saved: Data\1\21.jpg
Saved: Data\1\22.jpg
Saved: Data\1\23.jpg
Saved: Data\1\24.jpg
Saved: Data\1\25.jpg
Saved: Data\1\26.jpg
Saved: Data\1\27.jpg
Saved: Data\1\28.jpg
Saved: Data\1\29.jpg
Saved: Data\1\30.jpg
Saved: Data\1\31.jpg
Saved: Data\1\32.jpg
Saved: Data\1\33.jpg
Saved: Data\1\34.jpg
Saved: Data\1\35.jpg
Saved: Data\1\36.jpg
Saved: Data\1\37.jpg
Saved: Data\1\38.jpg
Saved: Data\1\39.jpg
Saved: Data\1\40.jpg
Saved: Data\1\41.jpg
Saved: Data\1\42.jpg
Saved: Data\1\43.jpg
Saved: Data\1\44.jpg
Saved: Data\1\45.jpg
Saved: Data\1\46.jpg
Saved: Data\1\47.jpg
Saved: Data\1\48.jpg
S